In [333]:
import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [334]:
import matplotlib.pyplot as plt
import  numpy as np
import pandas as pd
import random
import seaborn as sns

In [335]:
! pip install catboost

In [336]:
!pip install xgboost

In [337]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, log_loss
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
import catboost as cat_
import seaborn as sns
import lightgbm as lgb

In [338]:
# load datasets
train = pd.read_csv('C:/Users/USER/Downloads/Train.csv')
test  = pd.read_csv('C:/Users/USER/Downloads/Test.csv')
sample  = pd.read_csv('C:/Users/USER/Downloads/SampleSubmission.csv')

In [339]:
train.head(20)

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0000,0.0000,0.0,10689720.0,252072.0,4272776.0,...,0.392854,2.02,0.711632,0.0,0.0,charge,NaN,1.129518,0.044335,no
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0000,0.0000,0.0,898979.0,497531.0,9073814.0,...,0.314281,8.08,0.183584,NaN,0.0,charge,349.805730,1.620483,0.322436,no
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0000,6.2820,NaN,956940.0,NaN,192944.0,...,0.162965,18.18,0.791136,0.0,0.0,charge,NaN,1.513370,0.011640,yes
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0000,0.0000,192166.0,3044703.0,385499.0,3986472.0,...,0.488884,2.02,0.685168,NaN,0.0,charge,89.940100,0.664452,0.082729,no
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0000,0.0000,1556.0,214728.0,214728.0,1284089.0,...,0.275000,12.12,0.438168,0.0,0.0,charge,97.887502,1.427891,0.045630,no
5,Apcnt_1000020,3724.0,NaN,0.0000,0.0000,0.0000,192944.0,9244585.0,6761209.0,67963357.0,...,0.030558,12.12,0.382960,NaN,0.0,lending,NaN,0.000000,NaN,no
6,Apcnt_1000024,3350.0,0.24205,0.3132,0.0000,2.8188,NaN,580388.0,NaN,96472.0,...,0.660000,1.01,0.513504,NaN,0.0,charge,NaN,2.146512,0.004390,yes
7,Apcnt_1000028,3416.0,0.87020,0.0570,0.0000,0.0000,17505.0,186331.0,186331.0,1484813.0,...,0.137500,3.03,0.635960,NaN,0.0,charge,155.295994,0.946074,0.128389,no
8,Apcnt_1000036,3376.0,0.29180,0.5586,0.0000,0.0000,18283.0,1105149.0,18283.0,115533.0,...,0.471427,4.04,0.410880,0.0,0.0,charge,NaN,0.641601,0.009990,no
9,Apcnt_1000040,3310.0,0.24395,0.1562,0.0000,0.0000,NaN,0.0,NaN,NaN,...,0.825000,1.01,0.800000,NaN,NaN,charge,NaN,0.000000,NaN,yes


In [340]:
train.drop(['form_field48', 'form_field49','form_field47'], inplace=True, axis=1)

In [341]:
test.drop(['form_field48', 'form_field49','form_field47'], inplace=True, axis=1)

In [342]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [343]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56000 entries, 0 to 55999
Data columns (total 49 columns):
Applicant_ID      56000 non-null object
form_field1       53471 non-null float64
form_field2       52156 non-null float64
form_field3       55645 non-null float64
form_field4       55645 non-null float64
form_field5       55645 non-null float64
form_field6       42640 non-null float64
form_field7       50837 non-null float64
form_field8       42640 non-null float64
form_field9       47992 non-null float64
form_field10      55645 non-null float64
form_field11      24579 non-null float64
form_field12      46105 non-null float64
form_field13      50111 non-null float64
form_field14      56000 non-null int64
form_field15      33525 non-null float64
form_field16      42964 non-null float64
form_field17      44849 non-null float64
form_field18      45598 non-null float64
form_field19      55996 non-null float64
form_field20      55645 non-null float64
form_field21      40146 non-null 

In [344]:
train = reduce_mem_usage(train, verbose=True)

Mem. usage decreased to  7.69 Mb (63.3% reduction)


In [345]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56000 entries, 0 to 55999
Data columns (total 49 columns):
Applicant_ID      56000 non-null object
form_field1       53471 non-null float16
form_field2       52156 non-null float16
form_field3       55645 non-null float16
form_field4       55645 non-null float16
form_field5       55645 non-null float16
form_field6       42640 non-null float32
form_field7       50837 non-null float32
form_field8       42640 non-null float32
form_field9       47992 non-null float32
form_field10      55645 non-null float32
form_field11      24579 non-null float32
form_field12      46105 non-null float32
form_field13      50111 non-null float32
form_field14      56000 non-null int64
form_field15      33525 non-null float32
form_field16      42964 non-null float16
form_field17      44849 non-null float16
form_field18      45598 non-null float16
form_field19      55996 non-null float16
form_field20      55645 non-null float16
form_field21      40146 non-null 

In [346]:
non_number_train_columns = train.dtypes[train.dtypes == object].index.values
non_number_train_columns

array(['Applicant_ID', 'default_status'], dtype=object)

In [347]:
for col in non_number_train_columns:
    print(col, len(train[train[col]=='NA']))

Applicant_ID 0
default_status 0


In [348]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24000 entries, 0 to 23999
Data columns (total 48 columns):
Applicant_ID    24000 non-null object
form_field1     22890 non-null float64
form_field2     22291 non-null float64
form_field3     23854 non-null float64
form_field4     23854 non-null float64
form_field5     23854 non-null float64
form_field6     18396 non-null float64
form_field7     21769 non-null float64
form_field8     18396 non-null float64
form_field9     20600 non-null float64
form_field10    23853 non-null float64
form_field11    10602 non-null float64
form_field12    19817 non-null float64
form_field13    21537 non-null float64
form_field14    24000 non-null int64
form_field15    14408 non-null float64
form_field16    18526 non-null float64
form_field17    19305 non-null float64
form_field18    19631 non-null float64
form_field19    24000 non-null float64
form_field20    23853 non-null float64
form_field21    17293 non-null float64
form_field22    15276 non-null float

In [349]:
test = reduce_mem_usage(test, verbose=True)

Mem. usage decreased to  3.11 Mb (64.6% reduction)


In [350]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24000 entries, 0 to 23999
Data columns (total 48 columns):
Applicant_ID    24000 non-null object
form_field1     22890 non-null float16
form_field2     22291 non-null float16
form_field3     23854 non-null float16
form_field4     23854 non-null float16
form_field5     23854 non-null float16
form_field6     18396 non-null float32
form_field7     21769 non-null float32
form_field8     18396 non-null float32
form_field9     20600 non-null float32
form_field10    23853 non-null float32
form_field11    10602 non-null float32
form_field12    19817 non-null float32
form_field13    21537 non-null float32
form_field14    24000 non-null int64
form_field15    14408 non-null float32
form_field16    18526 non-null float16
form_field17    19305 non-null float16
form_field18    19631 non-null float16
form_field19    24000 non-null float16
form_field20    23853 non-null float16
form_field21    17293 non-null float16
form_field22    15276 non-null float

In [351]:
test.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field38,form_field39,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field50
0,Apcnt_1000032,3236.0,0.348633,10.203125,0.000000,0.0,418564.0,418564.0,418564.0,540710.0,...,14.0,0.0,NaN,NaN,0.825195,1.009766,0.799805,NaN,0.0,0.011221
1,Apcnt_1000048,3284.0,1.273438,2.960938,9.023438,0.0,0.0,9858816.0,49014.0,1510098.0,...,14.0,2.0,NaN,18.84375,0.507812,4.039062,0.623047,1.0,0.0,0.043525
2,Apcnt_1000052,NaN,0.275146,0.059998,0.000000,0.0,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN
3,Apcnt_1000076,3232.0,0.285156,2.802734,0.000000,0.0,0.0,473802.0,473802.0,1724437.0,...,26.0,0.0,NaN,NaN,0.916504,2.019531,0.464111,NaN,NaN,0.104029
4,Apcnt_1000080,3466.0,2.095703,0.832031,2.517578,0.0,19839.0,1150662.0,1150662.0,7860523.0,...,20.0,0.0,NaN,NaN,0.234009,23.234375,0.726562,0.0,0.0,0.163124


In [352]:
non_number_test_columns = test.dtypes[test.dtypes == object].index.values
non_number_test_columns

array(['Applicant_ID'], dtype=object)

In [353]:
train = train.fillna(train.mean())
test = test.fillna(test.mean())

In [354]:
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample 
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import RFE,SelectFromModel
from sklearn.model_selection import KFold,GridSearchCV,RandomizedSearchCV,train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline 
from sklearn.naive_bayes import GaussianNB
from scipy.stats import randint as sp_randint
from sklearn.ensemble import RandomForestClassifier ,ExtraTreesClassifier, AdaBoostClassifier,GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report,confusion_matrix,f1_score,roc_auc_score,roc_curve,accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [355]:
X = train.drop(['default_status','Applicant_ID'], axis=1)  
y = train['default_status']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.15, stratify=y, random_state=123)

In [356]:
LR = LogisticRegression(random_state=0)
LR.fit(X_train, y_train)
y_pred = LR.predict(X_test)
print('Accuracy:', accuracy_score(y_pred,y_test))

cv_scores = cross_val_score(LR, X, y, cv=5)
print()
print(classification_report(y_test, y_pred))
print()
print("Average 5-Fold CV Score: {}".format(round(np.mean(cv_scores),4)),
      ", Standard deviation: {}".format(round(np.std(cv_scores),4)))

LogisticRegression(random_state=0)

Accuracy: 0.7540476190476191

              precision    recall  f1-score   support

          no       0.76      0.99      0.86      6343
         yes       0.42      0.01      0.02      2057

    accuracy                           0.75      8400
   macro avg       0.59      0.50      0.44      8400
weighted avg       0.67      0.75      0.65      8400


Average 5-Fold CV Score: 0.7541 , Standard deviation: 0.0009


In [357]:
sample['default_status'] = y_test
sample.to_csv('SampleSubmission.csv',index=False)

In [359]:
sample.head(5)

,Applicant_ID,default_status
0,Apcnt_1000032,NaN
1,Apcnt_1000048,NaN
2,Apcnt_1000052,yes
3,Apcnt_1000076,NaN
4,Apcnt_1000080,NaN


In [ ]:
submission=pd.DataFrame({ 'Applicant_ID': test['Applicant_ID'],'default_status': predictions})

model.score(X_test, y_test)

predictions = model.predict_proba(X_test)[:,1]

submission=pd.DataFrame({ 'Applicant_ID': test['Applicant_ID'],'default_status': predictions})

submission = pd.DataFrame(submission)

submission.to_csv('first_submission.csv', index=False)